## <font color=blue> Target (ground truth): NC county deaths

Need ...
To figure out what the "naive" answer is. It would be the county ratio multiplied by the state prediction. First, do this for 2021.




In [1]:
import pandas as pd

df = pd.read_csv('raw_data/nyt_us-counties-2021.csv')
# display(df.head(), len(df))

df.columns = map(lambda x: str(x).capitalize(), df.columns)

df = df[df.State=='North Carolina']
df = df.drop(['Cases','State'],axis=1)
df = df[df.County!="Unknown"]

## Re-format

df = df.rename({'Deaths':'Target_deaths'},axis=1)

df['Target_deaths'] = df['Target_deaths'].astype('Int64')
df['Fips'] = df['Fips'].astype('Int64').astype('str')

df = df.reset_index(drop=True)

df_county_target = df
df_county_target

,Date,County,Fips,Target_deaths
0,2021-01-01,Alamance,37001,117
1,2021-01-01,Alexander,37003,30
2,2021-01-01,Alleghany,37005,2
3,2021-01-01,Anson,37007,26
4,2021-01-01,Ashe,37009,27
...,...,...,...,...
36495,2021-12-31,Wayne,37191,332
36496,2021-12-31,Wilkes,37193,205
36497,2021-12-31,Wilson,37195,243
36498,2021-12-31,Yadkin,37197,90


In [49]:
df_county_target['Fips']

0        37001
1        37003
2        37005
3        37007
4        37009
         ...  
36495    37191
36496    37193
36497    37195
36498    37197
36499    37199
Name: Fips, Length: 36500, dtype: object

In [2]:
# dff_target = df_target[df_target.County=='Alamance']
# dff_target
# dff = dff.reset_index(drop=True)
# print(dff)

# running_tot = 0
# for i,row in dff.iterrows():
#     print(i,row.Target)

## <font color=blue> Prediction: Naive deaths per county.

<font color=blue> Get population of each county from covariates json. Just use whatever ones. 2019?

In [45]:
import json

with open('raw_data/county_covariates.json') as f:
    d = json.load(f)
print(len(d))

county_covs = []
for entry in d:
    if entry['state']=='NC':
        county_covs.append(entry)
print(len(county_covs))



# df['Pop'] = 0

df_pop = pd.DataFrame(columns=['Fips','Pop'])

for idx in range(len(county_covs)):
    pop = county_covs[idx]['population']['2019']
    fips = county_covs[idx]['fips']
    df_pop.loc[idx] = pd.Series({'Fips':str(fips), 'Pop':pop})
    
    
tot_pop = sum(df_pop.Pop)
tot_pop

df_pop['Pop_ratio'] = df_pop['Pop'].div(tot_pop)
df_pop

3142
100


,Fips,Pop,Pop_ratio
0,37037,74470,0.0071
1,37001,169509,0.016162
2,37057,167609,0.015981
3,37069,69685,0.006644
4,37155,130625,0.012455
...,...,...,...
95,37175,34385,0.003278
96,37171,71783,0.006844
97,37043,11231,0.001071
98,37055,37009,0.003529


In [48]:
df_pop['Fips']

0     37037
1     37001
2     37057
3     37069
4     37155
      ...  
95    37175
96    37171
97    37043
98    37055
99    37047
Name: Fips, Length: 100, dtype: object

<font color=blue> Get state predictions for each week (?) or whatever.

In [46]:
df = pd.read_csv('processed_data/nc_state_projections.csv')

# Re-format dataframe ...
df.columns = map(lambda x: str(x).capitalize(), df.columns)
df = df[df.Type=='point']
df = df[df.Target=='1 wk ahead cum death']
df = df.drop(['Target','Forecast_date','Quantile','Type','Location'],axis=1)

df = df.rename({'Target_end_date':'Date','Value':'Predicted_deaths'},axis=1)
df = df.reset_index(drop=True)
df_state_forecast = df

print(len(df_state_forecast))
df_state_forecast.head()

40


,Date,Predicted_deaths
0,2021-02-06,9846
1,2021-02-13,10485
2,2021-02-20,11231
3,2021-02-27,11285
4,2021-03-06,11494


<font color=blue> Take the state forecasts, and for each date, multiple by county ratio to get county forecasts ...

In [47]:
dates_pred = list(df_state_forecast.Date.unique())

dff_county_target = df_county_target[df_county_target.Date.isin(dates_pred)]
# dff_county_target

dfff = dff_county_target.join(df_pop, on='Fips')
dfff

ValueError: You are trying to merge on object and int64 columns for key 'Fips'. If you wish to proceed you should use pd.concat

In [42]:
df_pop

,Fips,Pop,Pop_ratio
0,37037,74470,0.0071
1,37001,169509,0.016162
2,37057,167609,0.015981
3,37069,69685,0.006644
4,37155,130625,0.012455
...,...,...,...
95,37175,34385,0.003278
96,37171,71783,0.006844
97,37043,11231,0.001071
98,37055,37009,0.003529


In [41]:
dff_county_target['Pred_deaths'] = 0

for d in dates_pred:
#     print(d, type(d))
#     state_pred = df_state_forecast[d]['Predicted_deaths']
    state_pred_on_date = df_state_forecast[df_state_forecast.Date==d]['Predicted_deaths'].values[0]
    print(d,state_pred_on_date)
    for fips in dff_county_target.Fips.unique():
        ratio = df_pop[(df_pop.Fips==fips) & (df_pop.)]
        print(fips, )
#     print(state_pred)

2021-02-06 9846
37001
37003
37005
37007
37009
37011
37013
37015
37017
37019
37021
37023
37025
37027
37029
37031
37033
37035
37037
37039
37041
37043
37045
37047
37049
37051
37053
37055
37057
37059
37061
37063
37065
37067
37069
37071
37073
37075
37077
37079
37081
37083
37085
37087
37089
37091
37093
37095
37097
37099
37101
37103
37105
37107
37109
37113
37115
37117
37111
37119
37121
37123
37125
37127
37129
37131
37133
37135
37137
37139
37141
37143
37145
37147
37149
37151
37153
37155
37157
37159
37161
37163
37165
37167
37169
37171
37173
37175
37177
37179
37181
37183
37185
37187
37189
37191
37193
37195
37197
37199
2021-02-13 10485
37001
37003
37005
37007
37009
37011
37013
37015
37017
37019
37021
37023
37025
37027
37029
37031
37033
37035
37037
37039
37041
37043
37045
37047
37049
37051
37053
37055
37057
37059
37061
37063
37065
37067
37069
37071
37073
37075
37077
37079
37081
37083
37085
37087
37089
37091
37093
37095
37097
37099
37101
37103
37105
37107
37109
37113
37115
37117
37111
37119
37121
3

/tmp/ipykernel_3592141/4004548564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff_county_target['Pred_deaths'] = 0


In [30]:
x = df_state_forecast[df_state_forecast.Date=='2021-12-25']['Predicted_deaths'].values[0]
x

19285

In [5]:
df_county_target

,Date,County,Fips,Target_deaths
0,2021-01-01,Alamance,37001,117
1,2021-01-01,Alexander,37003,30
2,2021-01-01,Alleghany,37005,2
3,2021-01-01,Anson,37007,26
4,2021-01-01,Ashe,37009,27
...,...,...,...,...
36495,2021-12-31,Wayne,37191,332
36496,2021-12-31,Wilkes,37193,205
36497,2021-12-31,Wilson,37195,243
36498,2021-12-31,Yadkin,37197,90


In [6]:
df_county_target[df_county_target.Fips=='37001'][100:120]

,Date,County,Fips,Target_deaths
10000,2021-04-11,Alamance,37001,252
10100,2021-04-12,Alamance,37001,253
10200,2021-04-13,Alamance,37001,254
10300,2021-04-14,Alamance,37001,254
10400,2021-04-15,Alamance,37001,261
10500,2021-04-16,Alamance,37001,263
10600,2021-04-17,Alamance,37001,263
10700,2021-04-18,Alamance,37001,263
10800,2021-04-19,Alamance,37001,262
10900,2021-04-20,Alamance,37001,262


In [7]:
df_pop, df_state_forecast, df_target

NameError: name 'df_target' is not defined

In [ ]:
dfff = dff[(dff.type=='point') & (dff.target=='1 wk ahead cum death')]
# len(dfff), 
dfff.head()
# len(dfff)

Note: There are predictions for only 40 weeks (days) ...
ALSO, they are cumulative. So need to match that with cum groun truth deaths.

# <font color=magenta> NEXT: Add a 'cum death' column to Target dataframe !!! this should be the "actual" target ... 